# Umsetzung der Serving-Schicht (Lambda-Architektur) mit Redshift

### Ziele:
- Batch- und Echzeit-Sicht kombinieren

### Authentifizierung

In [ ]:
import psycopg2

config = { 'dbname': 'lambda', 
           'user':'nelson',
           'pwd':'LambdaApp1',
           'host':'serving-view.ctgixhn76zcs.eu-central-1.redshift.amazonaws.com',
           'port':'5439'
         }
conn =  psycopg2.connect(dbname=config['dbname'], host=config['host'], 
                              port=config['port'], user=config['user'], 
                              password=config['pwd'])

### Creating the serving layer
- Daten aus der Batch-Sicht in die Serving-Sicht kopieren
- Serving-Sicht mit Daten aus der Echtzeit-Sicht ergänzen

In [ ]:
curs = conn.cursor()
curs.execute(""" 
    DROP TABLE IF EXISTS serving_view; 

    SELECT * INTO serving_view FROM batch_view

    UPDATE  serving_view
    SET
        count_id = count_id + speed_view."count(id)",
    FROM speed_view
    WHERE    serving_view.location = speed_view.location ;

    INSERT INTO serving_view
    SELECT * FROM speed_view
    WHERE speed_view.location 
    NOT IN (
        SELECT 
            DISTINCT location 
        FROM 
            serving_view
    )
  
""")
curs.close()
conn.commit()
conn.close()